Based on https://www.sqlservercentral.com/articles/solve-sudoku-with-t-sql-part-1-1

In [ ]:
select version();

version()
11.3.2-MariaDB-1:11.3.2+maria~ubu2204


In [ ]:
create database sudoku;

Query OK

In [ ]:
use sudoku;

Query OK

/* 81 long number table of row, column and block mapping */

In [ ]:
create table tGrid (CoordinateID int primary key, RowNum int, ColumnNum int, BlockNum int);

Query OK

In [ ]:
INSERT INTO tGrid
SELECT seq+1 AS CoordinateID,
       seq div 9 +1 AS RowNum,
       seq MOD 9 +1 AS ColumnNum,
               (seq div 27)*3 + 1 + ((seq div 3) MOD 3) AS BlockNum
FROM seq_0_to_80;

Query OK

In [ ]:
select * from tGrid order by CoordinateID limit 28;

CoordinateID,RowNum,ColumnNum,BlockNum
1,1,1,1
2,1,2,1
3,1,3,1
4,1,4,2
5,1,5,2
6,1,6,2
7,1,7,3
8,1,8,3
9,1,9,3
10,2,1,1


Table of what we've already solved

In [ ]:
CREATE OR REPLACE TABLE tFixed (CoordinateID INT PRIMARY KEY, Value CHAR(1));

Query OK

In [ ]:
set @PuzzleIn = '023780460000620000060304080001000534280000097439000100010205040000036000056018370';

Query OK

In [ ]:
INSERT INTO tFixed (CoordinateID, Value)
SELECT CoordinateID, SUBSTRING(@PuzzleIn, CoordinateID, 1) AS Value
FROM tGrid
WHERE SUBSTRING(@PuzzleIn, CoordinateID, 1) != '0';

Query OK

In [ ]:
CREATE OR REPLACE TABLE tOption (CoordinateID INT, Value INT, Key (CoordinateID));

Query OK

All possible values

In [ ]:
SELECT CoordinateID, b.seq FROM tGrid AS a CROSS JOIN seq_1_to_9 AS b;

CoordinateID,seq
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


All possible values, excluding known values that are fixed

In [ ]:
SELECT CoordinateID,
       b.seq
FROM tGrid AS a
LEFT JOIN tFixed f USING (CoordinateID)
CROSS JOIN seq_1_to_9 AS b
WHERE f.CoordinateID IS NULL;

CoordinateID,seq
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
6,1


Same query using subquery

In [ ]:
SELECT CoordinateID,
       b.seq
FROM tGrid AS a
CROSS JOIN seq_1_to_9 AS b
WHERE NOT EXISTS
    (SELECT 1
     FROM tFixed c
     WHERE a.CoordinateID = c.CoordinateID );

CoordinateID,seq
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
6,1


In addition to exluding the fixed values we know about, exclude same value on the same row

AND NOT EXISTS (
  SELECT 1 FROM tFixed c
  INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
  WHERE a.RowNum = d.RowNum AND b.seq = c.Value
)

And same value on the same column

AND NOT EXISTS (
  SELECT 1 FROM tFixed c
  INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
  WHERE a.ColumnNum = d.ColumnNum AND b.seq = c.Value
)

And same value in the same grid

AND NOT EXISTS (
  SELECT 1 FROM tFixed c
  INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
  WHERE a.BlockNum = d.BlockNum AND b.seq = c.Value
)

And to this list of options append the fixed values we had

UNION
SELECT CoordinateID, Value FROM tFixed;

Resulting query

In [ ]:
INSERT INTO tOption (CoordinateID, Value)
SELECT CoordinateID,
       seq
FROM tGrid AS a
CROSS JOIN seq_1_to_9 AS b
WHERE NOT EXISTS
    (SELECT 1
     FROM tFixed c
     WHERE a.CoordinateID = c.CoordinateID )
  AND NOT EXISTS
    (SELECT 1
     FROM tFixed c
     INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
     WHERE a.RowNum = d.RowNum
       AND b.seq = c.Value )
  AND NOT EXISTS
    (SELECT 1
     FROM tFixed c
     INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
     WHERE a.ColumnNum = d.ColumnNum
       AND b.seq = c.Value )
  AND NOT EXISTS
    (SELECT 1
     FROM tFixed c
     INNER JOIN tGrid d ON c.CoordinateID = d.CoordinateID
     WHERE a.BlockNum = d.BlockNum
       AND b.seq = c.Value )
UNION
SELECT CoordinateID,
       Value
FROM tFixed;

Query OK

Duplicates exist

In [ ]:
select * from tOption order by CoordinateID limit 10;

CoordinateID,Value
1,1
1,5
1,9
2,2
3,3
4,7
5,8
6,1
6,9
7,4


So with 81 Coordinates (9 x 9 grid), cross join them all. Use SQL to generate SQL

In [ ]:
SELECT @SelectQuery := GROUP_CONCAT(' (SELECT Value AS `',
                                    CoordinateID,
                                    '` FROM tOption WHERE CoordinateID = ',
                                    CoordinateID,
                                    ') AS `',
                                    CoordinateID,
                                    '`'
                                    SEPARATOR ' CROSS JOIN ')
FROM
  (SELECT DISTINCT CoordinateID
   FROM tOption) AS b;

"@SelectQuery := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID,"


In [ ]:
SELECT @ExclusionQuery := GROUP_CONCAT('`', l.CoordinateID, '` <> `', r.CoordinateID, '`' SEPARATOR ' AND ')
FROM tGrid l
CROSS JOIN tGrid r
WHERE (l.RowNum = r.RowNum
       OR l.ColumnNum = r.ColumnNum
       OR l.BlockNum = r.BlockNum)
  AND r.CoordinateID > l.CoordinateID;

"@ExclusionQuery := GROUP_CONCAT('`', l.CoordinateID, '` <> `', r.CoordinateID, '`' SEPARATOR ' AND ')"


In [ ]:
select @columns := group_concat('`', seq, '`' SEPARATOR ',') from seq_1_to_81;

"@columns := group_concat('`', seq, '`' SEPARATOR ',')"
"`1`,`2`,`3`,`4`,`5`,`6`,`7`,`8`,`9`,`10`,`11`,`12`,`13`,`14`,`15`,`16`,`17`,`18`,`19`,`20`,`21`,`22`,`23`,`24`,`25`,`26`,`27`,`28`,`29`,`30`,`31`,`32`,`33`,`34`,`35`,`36`,`37`,`38`,`39`,`40`,`41`,`42`,`43`,`44`,`45`,`46`,`47`,`48`,`49`,`50`,`51`,`52`,`53`,`54`,`55`,`56`,`57`,`58`,`59`,`60`,`61`,`62`,`63`,`64`,`65`,`66`,`67`,`68`,`69`,`70`,`71`,`72`,`73`,`74`,`75`,`76`,`77`,`78`,`79`,`80`,`81`"


In [ ]:
EXECUTE IMMEDIATE concat('SELECT ROW_NUMBER() OVER (ORDER BY `1` ASC) AS SolutionID, ',
                         @columns,
                         ' FROM ( SELECT * FROM',
                         @SelectQuery,
                         ' WHERE ',
                         @ExclusionQuery,
                        ') AS x');

Too many tables; MariaDB can only use 61 tables in a join



In [ ]:
SELECT @SelectQuery50 := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID, ') AS `', CoordinateID, '`' SEPARATOR ' CROSS JOIN ')
FROM
  (SELECT seq AS CoordinateID
   FROM seq_1_to_50) b;

"@SelectQuery50 := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID, ') AS `', CoordinateID, '`' SEPARATOR ' CROSS JOIN ')"
(SELECT Value AS `1` FROM tOption WHERE CoordinateID = 1) AS `1` CROSS JOIN (SELECT Value AS `2` FROM tOption WHERE CoordinateID = 2) AS `2` CROSS JOIN (SELECT Value AS `3` FROM tOption WHERE CoordinateID = 3) AS `3` CROSS JOIN (SELECT Value AS `4` FROM tOption WHERE CoordinateID = 4) AS `4` CROSS JOIN (SELECT Value AS `5` FROM tOption WHERE CoordinateID = 5) AS `5` CROSS JOIN (SELECT Value AS `6` FROM tOption WHERE CoordinateID = 6) AS `6` CROSS JOIN (SELECT Value AS `7` FROM tOption WHERE CoordinateID = 7) AS `7` CROSS JOIN (SELECT Value AS `8` FROM tOption WHERE CoordinateID = 8) AS `8` CROSS JOIN (SELECT Value AS `9` FROM tOption WHERE CoordinateID = 9) AS `9` CROSS JOIN (SELECT Value AS `10` FROM tOption WHERE CoordinateID = 10) AS `10` CROSS JOIN (SELECT Value AS `11` FROM tOption WHERE CoordinateID = 11) AS `11` CROSS JOIN (SELECT Value AS `12` FROM tOption WHERE CoordinateID = 12) AS `12` CROSS JOIN (SELECT Value AS `13` FROM tOption WHERE CoordinateID = 13) AS `13` CROSS JOIN (SELECT Value AS `14` FROM tOption WHERE CoordinateID = 14) AS `14` CROSS JOIN (SELECT Value AS `15` FROM tOption WHERE CoordinateID = 15) AS `15` CROSS JOIN (SELECT Value AS `16` FROM tOption WHERE CoordinateID = 16) AS `16` CROSS JOIN (SELECT Value AS `17` FROM tOption WHERE CoordinateID = 17) AS `17` CROSS JOIN (SELECT Value AS `18` FROM tOption WHERE CoordinateID = 18) AS `18` CROSS JOIN (SELECT Value AS `19` FROM tOption WHERE CoordinateID = 19) AS `19` CROSS JOIN (SELECT Value AS `20` FROM tOption WHERE CoordinateID = 20) AS `20` CROSS JOIN (SELECT Value AS `21` FROM tOption WHERE CoordinateID = 21) AS `21` CROSS JOIN (SELECT Value AS `22` FROM tOption WHERE CoordinateID = 22) AS `22` CROSS JOIN (SELECT Value AS `23` FROM tOption WHERE CoordinateID = 23) AS `23` CROSS JOIN (SELECT Value AS `24` FROM tOption WHERE CoordinateID = 24) AS `24` CROSS JOIN (SELECT Value AS `25` FROM tOption WHERE CoordinateID = 25) AS `25` CROSS JOIN (SELECT Value AS `26` FROM tOption WHERE CoordinateID = 26) AS `26` CROSS JOIN (SELECT Value AS `27` FROM tOption WHERE CoordinateID = 27) AS `27` CROSS JOIN (SELECT Value AS `28` FROM tOption WHERE CoordinateID = 28) AS `28` CROSS JOIN (SELECT Value AS `29` FROM tOption WHERE CoordinateID = 29) AS `29` CROSS JOIN (SELECT Value AS `30` FROM tOption WHERE CoordinateID = 30) AS `30` CROSS JOIN (SELECT Value AS `31` FROM tOption WHERE CoordinateID = 31) AS `31` CROSS JOIN (SELECT Value AS `32` FROM tOption WHERE CoordinateID = 32) AS `32` CROSS JOIN (SELECT Value AS `33` FROM tOption WHERE CoordinateID = 33) AS `33` CROSS JOIN (SELECT Value AS `34` FROM tOption WHERE CoordinateID = 34) AS `34` CROSS JOIN (SELECT Value AS `35` FROM tOption WHERE CoordinateID = 35) AS `35` CROSS JOIN (SELECT Value AS `36` FROM tOption WHERE CoordinateID = 36) AS `36` CROSS JOIN (SELECT Value AS `37` FROM tOption WHERE CoordinateID = 37) AS `37` CROSS JOIN (SELECT Value AS `38` FROM tOption WHERE CoordinateID = 38) AS `38` CROSS JOIN (SELECT Value AS `39` FROM tOption WHERE CoordinateID = 39) AS `39` CROSS JOIN (SELECT Value AS `40` FROM tOption WHERE CoordinateID = 40) AS `40` CROSS JOIN (SELECT Value AS `41` FROM tOption WHERE CoordinateID = 41) AS `41` CROSS JOIN (SELECT Value AS `42` FROM tOption WHERE CoordinateID = 42) AS `42` CROSS JOIN (SELECT Value AS `43` FROM tOption WHERE CoordinateID = 43) AS `43` CROSS JOIN (SELECT Value AS `44` FROM tOption WHERE CoordinateID = 44) AS `44` CROSS JOIN (SELECT Value AS `45` FROM tOption WHERE CoordinateID = 45) AS `45` CROSS JOIN (SELECT Value AS `46` FROM tOption WHERE CoordinateID = 46) AS `46` CROSS JOIN (SELECT Value AS `47` FROM tOption WHERE CoordinateID = 47) AS `47` CROSS JOIN (SELECT Value AS `48` FROM tOption WHERE CoordinateID = 48) AS `48` CROSS JOIN (SELECT Value AS `49` FROM tOption WHERE CoordinateID = 4

In [ ]:
SELECT @ExclusionQuery50 := GROUP_CONCAT('`', l.CoordinateID, '` <> `', r.CoordinateID, '`' SEPARATOR ' AND ')
FROM tGrid l
CROSS JOIN tGrid r
WHERE (l.RowNum = r.RowNum
       OR l.ColumnNum = r.ColumnNum
       OR l.BlockNum = r.BlockNum)
  AND r.CoordinateID > l.CoordinateID
  AND r.CoordinateID <= 50
  AND l.CoordinateID <= 50;

"@ExclusionQuery50 := GROUP_CONCAT('`', l.CoordinateID, '` <> `', r.CoordinateID, '`' SEPARATOR ' AND ')"


In [ ]:
EXECUTE IMMEDIATE concat('create or replace table tOption50( CoordinateID int, key (CoordinateID))  select * FROM ',
                         @SelectQuery50,
                        ' WHERE ',
                        @ExclusionQuery50);

Query OK

In [ ]:
SELECT @SelectQuery81 := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID, ') AS `', CoordinateID, '`' SEPARATOR ' CROSS JOIN ')
FROM
  (SELECT seq AS CoordinateID
   FROM seq_51_to_81) b;

"@SelectQuery81 := GROUP_CONCAT(' (SELECT Value AS `', CoordinateID, '` FROM tOption WHERE CoordinateID = ', CoordinateID, ') AS `', CoordinateID, '`' SEPARATOR ' CROSS JOIN ')"
(SELECT Value AS `51` FROM tOption WHERE CoordinateID = 51) AS `51` CROSS JOIN (SELECT Value AS `52` FROM tOption WHERE CoordinateID = 52) AS `52` CROSS JOIN (SELECT Value AS `53` FROM tOption WHERE CoordinateID = 53) AS `53` CROSS JOIN (SELECT Value AS `54` FROM tOption WHERE CoordinateID = 54) AS `54` CROSS JOIN (SELECT Value AS `55` FROM tOption WHERE CoordinateID = 55) AS `55` CROSS JOIN (SELECT Value AS `56` FROM tOption WHERE CoordinateID = 56) AS `56` CROSS JOIN (SELECT Value AS `57` FROM tOption WHERE CoordinateID = 57) AS `57` CROSS JOIN (SELECT Value AS `58` FROM tOption WHERE CoordinateID = 58) AS `58` CROSS JOIN (SELECT Value AS `59` FROM tOption WHERE CoordinateID = 59) AS `59` CROSS JOIN (SELECT Value AS `60` FROM tOption WHERE CoordinateID = 60) AS `60` CROSS JOIN (SELECT Value AS `61` FROM tOption WHERE CoordinateID = 61) AS `61` CROSS JOIN (SELECT Value AS `62` FROM tOption WHERE CoordinateID = 62) AS `62` CROSS JOIN (SELECT Value AS `63` FROM tOption WHERE CoordinateID = 63) AS `63` CROSS JOIN (SELECT Value AS `64` FROM tOption WHERE CoordinateID = 64) AS `64` CROSS JOIN (SELECT Value AS `65` FROM tOption WHERE CoordinateID = 65) AS `65` CROSS JOIN (SELECT Value AS `66` FROM tOption WHERE CoordinateID = 66) AS `66` CROSS JOIN (SELECT Value AS `67` FROM tOption WHERE CoordinateID = 67) AS `67` CROSS JOIN (SELECT Value AS `68` FROM tOption WHERE CoordinateID = 68) AS `68` CROSS JOIN (SELECT Value AS `69` FROM tOption WHERE CoordinateID = 69) AS `69` CROSS JOIN (SELECT Value AS `70` FROM tOption WHERE CoordinateID = 70) AS `70` CROSS JOIN (SELECT Value AS `71` FROM tOption WHERE CoordinateID = 71) AS `71` CROSS JOIN (SELECT Value AS `72` FROM tOption WHERE CoordinateID = 72) AS `72` CROSS JOIN (SELECT Value AS `73` FROM tOption WHERE CoordinateID = 73) AS `73` CROSS JOIN (SELECT Value AS `74` FROM tOption WHERE CoordinateID = 74) AS `74` CROSS JOIN (SELECT Value AS `75` FROM tOption WHERE CoordinateID = 75) AS `75` CROSS JOIN (SELECT Value AS `76` FROM tOption WHERE CoordinateID = 76) AS `76` CROSS JOIN (SELECT Value AS `77` FROM tOption WHERE CoordinateID = 77) AS `77` CROSS JOIN (SELECT Value AS `78` FROM tOption WHERE CoordinateID = 78) AS `78` CROSS JOIN (SELECT Value AS `79` FROM tOption WHERE CoordinateID = 79) AS `79` CROSS JOIN (SELECT Value AS `80` FROM tOption WHERE CoordinateID = 80) AS `80` CROSS JOIN (SELECT Value AS `81` FROM tOption WHERE CoordinateID = 81) AS `81`


In [ ]:
EXECUTE IMMEDIATE concat('SELECT ROW_NUMBER() OVER (ORDER BY `1` ASC) AS SolutionID, ',
                         @columns,
                        ' FROM ( SELECT * FROM tOption50 CROSS JOIN ',
                        @SelectQuery81,
                        ' WHERE ',
                        @ExclusionQuery,
                        ') AS x');

SolutionID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81
1,5,2,3,7,8,9,4,6,1,8,9,4,6,2,1,7,5,3,1,6,7,3,5,4,2,8,9,6,7,1,8,9,2,5,3,4,2,8,5,1,4,3,6,9,7,4,3,9,5,6,7,1,2,8,3,1,8,2,7,5,9,4,6,7,4,2,9,3,6,8,1,5,9,5,6,4,1,8,3,7,2


In [ ]:
set @PuzzleIn = '023780460000620000060304080001000534280000097439000100010205040000036000056018370';







In [ ]:
set @PuzzleIn = '600050002010702040000346000084000590509000207032000480000165000020407060300090004';

In [ ]:
set @PuzzleIn = '800259004040010070000407000302080506580302091607040302000504000060090020700826009';


In [ ]:
set @PuzzleIn = '703200104054019380000500000070000805060000030308000090000001000035920460407008903';

In [ ]:
set @PuzzleIn = '700200008020405070004080300060508037009000800180607020001070600050903010600004003';

In [ ]:
set @PuzzleIn = '000400000004063200809000503090030005040658030600010070901000804005840100000007000';

In [ ]:
set @PuzzleIn = '007400803000206001000085700026000039004000100370000680908620000000108000630009200';


In [ ]:
set @PuzzleIn = '019600430000098000002005100098074001020000080700850620007500800000730000056001970';


In [ ]:
set @PuzzleIn = '280060079100007006070930080907000605008000700040000090090025060800600002650010047';

In [ ]:
set @PuzzleIn = '107608209840050036000000000300106004060000090200305007000000000430060052508209301';